In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp dcmm

# DCMM

> A Dynamic Count Mixture Model, or DCMM, is the combination of a Bernoulli and Poisson DGLM as described in [Berry and West (2019)](https://arxiv.org/pdf/1805.05232.pdf).

The DCMM is a combination of a Bernoulli and Poisson DGLM. The Bernoulli DGLM models the probability of the observation being zero. Conditional on a non-zero outcome, then the observation follows a Poisson distribution. This is useful for modeling time series with a greater number of zeros than expected under a Poisson distribution, which is frequently the case for low-valued count time series.

In more formal terms, a DCMM models observations $y_t$ as:
$$
\quad z_{t} \sim Bern(\pi_{t}) \quad \textrm{and}\quad y_{t} | z_{t}  = 
\begin{cases}
0, & \text{if } z_{t} = 0,\\
1 + x_{t}, \quad x_{t} \sim Pois(\mu_{t}), & \textrm{if}\ z_{t} = 1.
\end{cases}
$$

In [ ]:
#hide
#exporti
import numpy as np
from pybats.latent_factor_fxns import forecast_marginal_lf_dcmm, forecast_path_lf_dcmm
from pybats.dglm import bern_dglm, pois_dglm
from pybats.update import update_F
from scipy.special import expit

In [ ]:
#export
class dcmm:
    def __init__(self,
                 a0_bern = None,
                 R0_bern = None,
                 nregn_bern = 0,
                 ntrend_bern = 0,
                 nlf_bern = 0,
                 nhol_bern = 0,
                 seasPeriods_bern = [],
                 seasHarmComponents_bern = [],
                 deltrend_bern = 1, delregn_bern = 1,
                 delhol_bern = 1,
                 delseas_bern = 1, dellf_bern = 1,
                 a0_pois = None,
                 R0_pois = None,
                 nregn_pois = 0,
                 ntrend_pois = 0,
                 nlf_pois = 0,
                 nhol_pois = 0,
                 seasPeriods_pois = [],
                 seasHarmComponents_pois = [],
                 deltrend_pois = 1, delregn_pois = 1,
                 delhol_pois = 1,
                 delseas_pois = 1, dellf_pois = 1,
                 rho = 1,
                 interpolate=True,
                 adapt_discount=False):
        """
        :param a0_bern: Prior mean vector for bernoulli DGLM
        :param R0_bern: Prior covariance matrix for bernoulli DGLM
        :param nregn_bern: Number of regression components in bernoulli DGLM
        :param ntrend_bern: Number of trend components in bernoulli DGLM
        :param nlf_bern: Number of latent factor components in bernoulli DGLM
        :param seasPeriods_bern: List of periods of seasonal components in bernoulli DGLM
        :param seasHarmComponents_bern: List of harmonic components included for each period in bernoulli DGLM
        :param deltrend_bern: Discount factor on trend components in bernoulli DGLM
        :param delregn_bern: Discount factor on regression components in bernoulli DGLM
        :param delhol_bern: Discount factor on holiday component in bernoulli DGLM (currently deprecated)
        :param delseas_bern: Discount factor on seasonal components in bernoulli DGLM
        :param dellf_bern: Discount factor on latent factor components in bernoulli DGLM
        :param a0_pois: Prior mean vector for poisson DGLM
        :param R0_pois: Prior covariance matrix for poisson DGLM
        :param nregn_pois: Number of regression components in poisson DGLM
        :param ntrend_pois: Number of trend components in poisson DGLM
        :param nlf_pois: Number of latent factor components in poisson DGLM
        :param seasPeriods_pois: List of periods of seasonal components in poisson DGLM
        :param seasHarmComponents_pois: List of harmonic components included for each period in poisson DGLM
        :param deltrend_pois: Discount factor on trend components in poisson DGLM
        :param delregn_pois: Discount factor on regression components in poisson DGLM
        :param delhol_pois: Discount factor on holiday component in poisson DGLM (currently deprecated)
        :param delseas_pois: Discount factor on seasonal components in poisson DGLM
        :param dellf_pois: Discount factor on latent factor components in poisson DGLM
        :param rho: Discount factor for random effects extension in poisson DGLM (smaller rho increases variance)
        """

        self.bern_mod = bern_dglm(a0=a0_bern,
                                  R0=R0_bern,
                                  nregn=nregn_bern,
                                  ntrend=ntrend_bern,
                                  nlf=nlf_bern,
                                  nhol=nhol_bern,
                                  seasPeriods=seasPeriods_bern,
                                  seasHarmComponents=seasHarmComponents_bern,
                                  deltrend=deltrend_bern, delregn=delregn_bern,
                                  delhol=delhol_bern, delseas=delseas_bern,
                                  dellf=dellf_bern,
                                  interpolate=interpolate,
                                  adapt_discount=adapt_discount)

        self.pois_mod = pois_dglm(a0=a0_pois,
                                  R0=R0_pois,
                                  nregn=nregn_pois,
                                  ntrend=ntrend_pois,
                                  nlf=nlf_pois,
                                  nhol=nhol_pois,
                                  seasPeriods=seasPeriods_pois,
                                  seasHarmComponents=seasHarmComponents_pois,
                                  deltrend=deltrend_pois, delregn=delregn_pois,
                                  delhol=delhol_pois, delseas=delseas_pois,
                                  dellf=dellf_pois,
                                  rho=rho,
                                  interpolate=interpolate,
                                  adapt_discount=adapt_discount)

        self.t = 0


    # X is a list or tuple of length 2. The first component is data for the bernoulli DGLM, the next is for the Poisson DGLM.
    def update(self, y = None, X = None):
        X = self.make_pair(X)

        if y is None:
            self.bern_mod.update(y=y)
            self.pois_mod.update(y=y)
        elif y == 0:
            self.bern_mod.update(y = 0, X = X[0])
            self.pois_mod.update(y = np.nan, X = X[1])
        else: # only update beta model if we have significant uncertainty in the forecast
            # get the lower end forecast on the logit scale
            F = update_F(self.bern_mod, X[0], F=self.bern_mod.F.copy())
            ft, qt = self.bern_mod.get_mean_and_var(F, self.bern_mod.a, self.bern_mod.R)
            fcast_logit_lb = ft - np.sqrt(qt)
            # translate to a prod for a rough idea of whether we're already pretty confident for this forecast
            if expit(fcast_logit_lb) < 0.975:
                self.bern_mod.update(y=1, X = X[0])
            else:
                self.bern_mod.update(y=np.nan, X=X[0])
            self.pois_mod.update(y = y - 1, X = X[1]) # Shifted Y values in the Poisson DGLM

        self.t += 1

    def update_lf_sample(self, y = None, X = None, phi_samps = None, parallel=False):
        X = self.make_pair(X)
        phi_samps = self.make_pair(phi_samps)

        if y is None:
            self.bern_mod.update_lf_sample(y=y)
            self.pois_mod.update_lf_sample(y=y)
        elif y == 0:
            self.bern_mod.update_lf_sample(y = 0, X = X[0], phi_samps = phi_samps[0], parallel = parallel)
            self.pois_mod.update_lf_sample(y = np.nan, X = X[1], phi_samps = phi_samps[1], parallel = parallel)
        else:
            self.bern_mod.update_lf_sample(y = 1, X = X[0], phi_samps = phi_samps[0], parallel = parallel)
            # Shifted Y values in the Poisson DGLM
            self.pois_mod.update_lf_sample(y =y - 1, X = X[1], phi_samps = phi_samps[1], parallel = parallel)

        self.t += 1

    def update_lf_analytic(self, y = None, X = None, phi_mu = None, phi_sigma = None):
        X = self.make_pair(X)
        phi_mu = self.make_pair(phi_mu)
        phi_sigma = self.make_pair(phi_sigma)

        if y is None:
            self.bern_mod.update_lf_analytic(y=y)
            self.pois_mod.update_lf_analytic(y=y)
        elif y == 0:
            self.bern_mod.update_lf_analytic(y = 0, X = X[0], phi_mu = phi_mu[0], phi_sigma = phi_sigma[0])
            self.pois_mod.update_lf_analytic(y = np.nan, X = X[1], phi_mu = phi_mu[1], phi_sigma = phi_sigma[1])
        else:
            self.bern_mod.update_lf_analytic(y = 1, X = X[0], phi_mu = phi_mu[0], phi_sigma = phi_sigma[0])
            # Shifted Y values in the Poisson DGLM
            self.pois_mod.update_lf_analytic(y =y - 1, X = X[1], phi_mu = phi_mu[1], phi_sigma = phi_sigma[1])

        self.t += 1

    def forecast_marginal(self, k, X = None, nsamps = 1, mean_only = False, state_mean_var = False):
        X = self.make_pair(X)

        if mean_only:
            mean_bern = self.bern_mod.forecast_marginal(k, X[0], nsamps, mean_only)
            mean_pois = self.pois_mod.forecast_marginal(k, X[1], nsamps, mean_only)
            return mean_bern * (mean_pois + 1)
        elif state_mean_var:
            mv_bern = self.bern_mod.forecast_marginal(k, X[0], state_mean_var = state_mean_var)
            mv_pois = self.pois_mod.forecast_marginal(k, X[1], state_mean_var = state_mean_var)
            return mv_bern, mv_pois
        else:
            samps_bern = self.bern_mod.forecast_marginal(k, X[0], nsamps)
            samps_pois = self.pois_mod.forecast_marginal(k, X[1], nsamps) + np.ones([nsamps]) # Shifted Y values in the Poisson DGLM
            return samps_bern * samps_pois

    def forecast_marginal_lf_analytic(self, k, X = None, phi_mu = None, phi_sigma = None, nsamps = 1, mean_only = False, state_mean_var = False):
        X = self.make_pair(X)
        phi_mu = self.make_pair(phi_mu)
        phi_sigma = self.make_pair(phi_sigma)

        if mean_only:
            mean_bern = self.bern_mod.forecast_marginal_lf_analytic(k, X[0], phi_mu[0], phi_sigma[0], nsamps, mean_only)
            mean_pois = self.pois_mod.forecast_marginal_lf_analytic(k, X[1], phi_mu[1], phi_sigma[1], nsamps, mean_only)
            return np.array([[mean_bern * (mean_pois + 1)]])
        elif state_mean_var:
            mv_bern = self.bern_mod.forecast_marginal_lf_analytic(k, X[0], phi_mu[0], phi_sigma[0], state_mean_var = state_mean_var)
            mv_pois = self.pois_mod.forecast_marginal_lf_analytic(k, X[1], phi_mu[1], phi_sigma[1], state_mean_var = state_mean_var)
            return mv_bern, mv_pois
        else:
            samps_bern = self.bern_mod.forecast_marginal_lf_analytic(k, X[0], phi_mu = phi_mu[0], phi_sigma = phi_sigma[0], nsamps = nsamps)
            samps_pois = self.pois_mod.forecast_marginal_lf_analytic(k, X[1], phi_mu = phi_mu[1], phi_sigma = phi_sigma[1], nsamps = nsamps) + np.ones([nsamps]) # Shifted Y values in the Poisson DGLM
            return samps_bern * samps_pois

    def forecast_marginal_lf_analytic_new(self, k, X = None, phi_mu = None, phi_sigma = None, nsamps = 1, mean_only = False, state_mean_var = False):
        X = self.make_pair(X)
        phi_mu = self.make_pair(phi_mu)
        phi_sigma = self.make_pair(phi_sigma)

        if mean_only:
            mean_bern = self.bern_mod.forecast_marginal_lf_analytic(k, X[0], phi_mu[0], phi_sigma[0], nsamps, mean_only)
            mean_pois = self.pois_mod.forecast_marginal_lf_analytic(k, X[1], phi_mu[1], phi_sigma[1], nsamps, mean_only)
            return np.array([[mean_bern * (mean_pois + 1)]])
        elif state_mean_var:
            mv_bern = self.bern_mod.forecast_marginal_lf_analytic(k, X[0], phi_mu[0], phi_sigma[0], state_mean_var = state_mean_var)
            mv_pois = self.pois_mod.forecast_marginal_lf_analytic(k, X[1], phi_mu[1], phi_sigma[1], state_mean_var = state_mean_var)
            return mv_bern, mv_pois
        else:
            return forecast_marginal_lf_dcmm(self, k, X[0], phi_mu[0], phi_sigma[0], nsamps=nsamps)

    def forecast_marginal_lf_sample(self, k, X = None, phi_samps = None, nsamps = 1, mean_only = False):
        X = self.make_pair(X)
        phi_samps = self.make_pair(phi_samps)

        samps_bern = self.bern_mod.forecast_marginal_lf_sample(k, X[0], phi_samps[0], mean_only)
        samps_pois = self.pois_mod.forecast_marginal_lf_sample(k, X[1], phi_samps[1], mean_only) + np.ones([nsamps]) # Shifted Y values in the Poisson DGLM
        return samps_bern * samps_pois

    def forecast_path_lf_sample(self, k, X = None, phi_samps=None, nsamps = 1):
        X = self.make_pair(X)
        phi_samps = self.make_pair(phi_samps)

        samps_bern = self.bern_mod.forecast_path_lf_sample(k, X[0], phi_samps[0], nsamps)
        samps_pois = self.pois_mod.forecast_path_lf_sample(k, X[1], phi_samps[1], nsamps) + np.ones([nsamps, k]) # Shifted Y values in the Poisson DGLM
        return samps_bern * samps_pois

    def forecast_path(self, k, X = None, nsamps = 1):
        X = self.make_pair(X)

        samps_bern = self.bern_mod.forecast_path(k, X[0], nsamps)
        samps_pois = self.pois_mod.forecast_path(k, X[1], nsamps) + np.ones([nsamps, k]) # Shifted Y values in the Poisson DGLM
        return samps_bern * samps_pois

    def forecast_path_copula(self, k, X = None, nsamps = 1, **kwargs):
        X = self.make_pair(X)

        samps_bern = self.bern_mod.forecast_path_copula(k, X[0], nsamps, **kwargs)
        samps_pois = self.pois_mod.forecast_path_copula(k, X[1], nsamps, **kwargs) + np.ones([nsamps, k]) # Shifted Y values in the Poisson DGLM
        return samps_bern * samps_pois

    def forecast_path_lf_copula(self, k, X = None, phi_mu = None, phi_sigma = None, phi_psi = None, nsamps = 1, **kwargs):
        X = self.make_pair(X)
        if k == 2 and isinstance(phi_mu, (list, tuple)):
            if not isinstance(phi_mu[0], (list, tuple)):
                phi_mu = (phi_mu, phi_mu)
                phi_sigma = (phi_sigma, phi_sigma)
                phi_psi = (phi_psi, phi_psi)
        else:
            phi_mu = self.make_pair(phi_mu)
            phi_sigma = self.make_pair(phi_sigma)
            phi_psi = self.make_pair(phi_psi)

        samps_bern = self.bern_mod.forecast_path_lf_copula(k, X[0], phi_mu = phi_mu[0], phi_sigma = phi_sigma[0], phi_psi = phi_psi[0], nsamps = nsamps, **kwargs)
        samps_pois = self.pois_mod.forecast_path_lf_copula(k, X[1], phi_mu = phi_mu[1], phi_sigma = phi_sigma[1], phi_psi = phi_psi[1], nsamps = nsamps, **kwargs) + np.ones([nsamps, k]) # Shifted Y values in the Poisson DGLM
        return samps_bern * samps_pois

    def forecast_path_lf_copula_new(self, k, X = None, phi_mu = None, phi_sigma = None, phi_psi = None, nsamps = 1, **kwargs):
        X = self.make_pair(X)
        if k == 2 and isinstance(phi_mu, (list, tuple)):
            if not isinstance(phi_mu[0], (list, tuple)):
                phi_mu = (phi_mu, phi_mu)
                phi_sigma = (phi_sigma, phi_sigma)
                phi_psi = (phi_psi, phi_psi)
        else:
            phi_mu = self.make_pair(phi_mu)
            phi_sigma = self.make_pair(phi_sigma)
            phi_psi = self.make_pair(phi_psi)

        return forecast_path_lf_dcmm(self, k, X[0], phi_mu[0], phi_sigma[0], phi_psi[0], nsamps=nsamps, **kwargs)


    def forecast_path_lf_copula_density(self, y, k, X = None, phi_mu = None, phi_sigma = None, phi_psi = (None, None), nsamps = 1, **kwargs):
        X = self.make_pair(X)
        phi_mu = self.make_pair(phi_mu)
        phi_sigma = self.make_pair(phi_sigma)
        phi_psi = self.make_pair(phi_psi)

        z = np.zeros([k])
        y = y.reshape(-1)
        z[y > 0] = 1
        logdens_bern = self.bern_mod.forecast_path_lf_copula(k, X[0], phi_mu = phi_mu[0], phi_sigma = phi_sigma[0], phi_psi = phi_psi[0], nsamps = nsamps, y = z, **kwargs)
        # Shifted Y values in the Poisson DGLM
        y = y - 1
        y = y.astype('float')
        # 0's in the original data (now -1's) are considered 'missing by the Poisson model
        y[y < 0] = np.nan
        logdens_pois = self.pois_mod.forecast_path_lf_copula(k, X[1], phi_mu = phi_mu[1], phi_sigma = phi_sigma[1], phi_psi = phi_psi[1], nsamps = nsamps, y = y, **kwargs)
        return logdens_bern, logdens_pois

    def forecast_state_mean_and_var(self, k = 1, X = None):
        mean_var_bern = self.bern_mod.forecast_state_mean_and_var(k, X[0])
        mean_var_pois = self.pois_mod.forecast_state_mean_and_var(k, X[1])
        return mean_var_bern, mean_var_pois

    def make_pair(self, x):
        if isinstance(x, (list, tuple)):
            if len(x) == 2:
                return x
            else:
                return (x, x)
        else:
            return (x, x)

A DCMM can be used in the same way as a DGLM, with the standard methods `dcmm.update`, `dcmm.forecast_marginal`, and `dcmm.forecast_path`. There are equivalent helper functions as well. A full analysis can be run with `analysis_dcmm`, and `define_dcmm` helps to initialize a DCMM. These helper functions assume that the same predictors `X` are used for the Bernoulli and Poisson DGLMs.

The only difference from using a standard `dglm` is that outside of `analysis_dcmm`, the update and forecast functions do not automatically recognize whether the DCMM includes latent factors or call a copula for path forecasting. This means that the modeler needs to be more explicit in calling the correct method, such as `dcmm.forecast_path_copula` for path forecasting with a copula.

A quick example of using `analysis_dcmm` to model simulated sales data follows. Another example with a DCMM can also be found [here](https://github.com/lavinei/pybats_nbdev/blob/master/examples/DCMM%20Latent%20Factor%20Example.ipynb).

In [ ]:
import pandas as pd
import numpy as np

from pybats.shared import load_sales_example2
from pybats.analysis import analysis_dcmm
from pandas.tseries.holiday import USFederalHolidayCalendar


data = load_sales_example2()
data.head()

,Sales,Price,Promotion
Date,,,
2014-06-01,15.0,1.11,0.0
2014-06-02,13.0,2.19,0.0
2014-06-03,6.0,0.23,0.0
2014-06-04,2.0,-0.05,1.0
2014-06-05,6.0,-0.14,0.0


In [ ]:
prior_length = 25   # Number of days of data used to set prior
k = 7               # Forecast horizon
rho = 0.5           # Random effect discount factor to increase variance of forecast distribution
forecast_samps = 1000  # Number of forecast samples to draw
forecast_start = pd.to_datetime('2018-01-01') # Date to start forecasting
forecast_end = pd.to_datetime('2018-05-01')   # Date to stop forecasting
holidays = USFederalHolidayCalendar.rules

In [ ]:
mod, samples = analysis_dcmm(data['Sales'].values, data[['Price', 'Promotion']].values,
                             k, forecast_start, forecast_end,
                             nsamps=forecast_samps,
                             prior_length=prior_length,
                             seasPeriods=[7], seasHarmComponents=[[1,2,3]],
                             dates=data.index, holidays=holidays,
                             rho=rho,
                             ret = ['model', 'forecast'])

beginning forecasting


Because the DCMM is effectively a container for a Poisson and a Bernoulli DGLM, we can access each of them individually. The coefficients in the Bernoulli DGLM affect the probability of a non-zero observation, and the coefficients in the Poisson DGLM impact the size of any non-zero observations. To illustrate, we'll take a look at the holiday coefficients in both DGLMs.

In [ ]:
pois_hol = mod.pois_mod.get_coef('hol')
bern_hol = mod.bern_mod.get_coef('hol')

coef = pd.DataFrame({'Holidays':[h.name for h in holidays],
                     'Pois Mean': pois_hol['Mean'],
                     'Pois Std Dev': pois_hol['Standard Deviation'],
                     'Bern Mean': bern_hol['Mean'],
                     'Bern Std Dev': bern_hol['Standard Deviation']}).round(2)
coef

,Holidays,Pois Mean,Pois Std Dev,Bern Mean,Bern Std Dev
Hol 1,New Years Day,-0.94,0.66,-0.78,1.29
Hol 2,Martin Luther King Jr. Day,0.20,0.43,0.08,1.41
Hol 3,Presidents Day,-0.27,0.46,0.27,1.39
Hol 4,Memorial Day,0.62,0.40,0.04,1.41
Hol 5,July 4th,1.10,0.43,0.04,1.41
Hol 6,Labor Day,0.21,0.43,0.03,1.41
Hol 7,Columbus Day,-0.04,0.45,0.21,1.39
Hol 8,Veterans Day,-0.15,0.48,0.03,1.41
Hol 9,Thanksgiving,0.14,0.43,0.16,1.39
Hol 10,Christmas,-1.97,1.11,-1.10,1.23


The largest negative coefficients are for Christmas and New Years Day, which means that they are more likely to have very low or $0$ sales.

The largest positive coefficients are for July 4th and Memorial day, which means that they are likely to have increased sales.

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_dglm.ipynb.
Converted 01_update.ipynb.
Converted 02_forecast.ipynb.
Converted 03_define_models.ipynb.
Converted 04_seasonal.ipynb.
Converted 05_analysis.ipynb.
Converted 06_conjugates.ipynb.
Converted 07_point_forecast.ipynb.
Converted 08_loss_functions.ipynb.
Converted 09_plot.ipynb.
Converted 10_shared.ipynb.
Converted 11_dcmm.ipynb.
Converted 12_dbcm.ipynb.
Converted 13_latent_factor.ipynb.
Converted 14_latent_factor_fxns.ipynb.
Converted index.ipynb.
